In [11]:
import pandas as pd
import re
from datetime import datetime
from feature_eng import *
import os 
import joblib
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [12]:

cols = [
"LASTNAME",
"FIRSTNAME",
"MIDDLENAME",
"NAMESUFFIX",
"RADDNUMBER",
"RHALFCODE",
"RPREDIRECTION",
"RSTREETNAME",
"RPOSTDIRECTION",
"RAPARTMENTTYPE",
"RAPARTMENT",
"RADDRNONSTD",
"RCITY",
"RZIP5",
"RZIP4",
"MAILADD1",
"MAILADD2",
"MAILADD3",
"MAILADD4",
"DOB",
"GENDER",
"ENROLLMENT",
"OTHERPARTY",
"COUNTYCODE",
"ED",
"LD",
"TOWNCITY",
"WARD",
"CD",
"SD",
"AD",
"LASTVOTERDATE",
"PREVYEARVOTED",
"PREVCOUNTY",
"PREVADDRESS",
"PREVNAME",
"COUNTYVRNUMBER",
"REGDATE",
"VRSOURCE",
"IDREQUIRED",
"IDMET",
"STATUS",
"REASONCODE",
"INACT_DATE",
"PURGE_DATE",
"SBOEID",
"VoterHistory"
]

,SBOEID,RZIP5,ED,LD,ENROLLMENT,REGDATE,STATUS,voterhistory,DOB
14,NY2927c2a9-fb57-,13790,20,9,BLK,19970808,A,2024 General Election(M);2022 General Election...,19650819
15,NYe3c21aaf-fcf9-,13760,30,2,DEM,19980227,A,2019 General Election(P);2015 General Election...,19300508
24,NYbb8374ed-e308-,13790,15,10,BLK,19951011,A,2008 Presidential Primary Election(A);2000 Pri...,19160309
46,NYfbfd679c-dbcc-,13850,24,5,DEM,19830513,A,2024 General Election(M);2022 Primary Election...,19461020
52,NYa31d09b6-521b-,13901,31,2,BLK,20240718,A,2024 Primary Election(M),19460623


In [27]:

path = "/Users/nickgutin/Desktop/projects/NYC-Voting-Analysis/Broome/synthetic_voter_data.csv"

current_voter_df = pd.read_csv(path, 
                 encoding='latin-1',  # or 'cp1252' if this doesn't work
                 names=cols,       # use our column names
                 quoting=1,          # handle the double quotes
                 quotechar='"',
                 index_col = False,  
                 na_values=[""]) 

# Rename VoterHistory column if needed

current_voter_df = current_voter_df.rename(columns={'VoterHistory': 'voterhistory'})

# Parse voter histories
needed_columns = ['SBOEID', 'RZIP5', 'ED', 'LD', 'ENROLLMENT', 'REGDATE', 'STATUS', 'voterhistory', 'DOB']
current_voter_df = current_voter_df[needed_columns].copy()



mayoral_election_dates = [
    '20251104',  # Projected 2025 mayoral election 
    '20211102',  # 2021 mayoral election
    '20171107',  # 2017 mayoral election
    '20131105'   # 2013 mayoral election
]

#calibrated_model = joblib.load(os.path.join('model', 'calibrated_lightgbm_model.joblib'))

X, _ = extract_clean_features_enhanced(current_voter_df, parse_voter_history, mayoral_election_dates)

/Library/Python/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (4,5,6,8,10,11,13,14,15,16,17,18,19,23,24,25,27,28,29,30,31,32,33,34,35,36,37,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Extracting clean features from 25001 voter records
Parsing voter histories...
Kept 25000 voters with valid parsed histories
Deriving all election dates from voter histories...
Derived 52 election dates
Mayoral election years: [2025, 2021, 2017, 2013]
Local election years: [2003, 2007, 2011, 2015, 2019, 2023, 2027]
Extracting features...
Calculating first election information...
Calculating normalized participation rates...
Calculating mayoral and local participation rates...
Setting target variable...
Calculating voting trends...
Feature extraction complete
Extracting behavior change features...


In [28]:
X

,SBOEID,RZIP5,ED,LD,ENROLLMENT,REGDATE,STATUS,voterhistory,DOB,parsed_history,...,participation_acceleration,avg_gap,gap_variability,longest_participation_streak,recent_pattern_change,participation_expansion,diversification_score,has_behavior_change,years_since_last_change,change_direction
1,NY830cbd57-96ea-,13795,8,4,OTH,20200201,A,2024 General Election(P);2024 Primary Election...,19870508,"[{'year': 2024, 'election_name': 'General Elec...",...,2.000000,1.333333,1.885618,1,1,0,1.00,1,2,1
2,NY766abd27-a5b3-,13903,22,2,REP,19821215,A,2024 Presidential Primary Election(P);2022 Gen...,19360213,"[{'year': 2024, 'election_name': 'Presidential...",...,1.361111,1.142857,0.989743,3,0,1,1.25,1,4,-1
3,NYe437a174-dc88-,13760,31,3,OTH,20200326,I,2024 General Election(A),19830720,"[{'year': 2024, 'election_name': 'General Elec...",...,0.000000,0.000000,0.000000,1,0,0,0.00,0,0,0
4,NY34fe613f-a3e3-,13905,14,9,DEM,19950108,A,2013 General Election(P);2004 Primary Election...,19711003,"[{'year': 2013, 'election_name': 'General Elec...",...,0.600000,6.000000,3.000000,1,1,1,2.00,1,13,1
5,NYa447f596-42ba-,13901,12,4,BLK,20100707,A,2022 Primary Election(A),19360222,"[{'year': 2022, 'election_name': 'Primary Elec...",...,0.000000,0.000000,0.000000,1,0,0,0.00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24996,NY278d9776-08e9-,13790,22,11,DEM,20200721,A,2024 General Election(P);2024 Primary Election...,19480115,"[{'year': 2024, 'election_name': 'General Elec...",...,-0.500000,1.000000,1.000000,1,1,0,1.00,0,0,0
24997,NYa1c30a0a-4f53-,13790,24,10,REP,20060520,A,2022 General Election(A);2020 General Election...,19420319,"[{'year': 2022, 'election_name': 'General Elec...",...,0.000000,3.200000,0.979796,1,1,0,1.00,1,5,1
24998,NY778fd0c2-6aad-,13901,32,14,DEM,20241106,A,2024 Presidential Primary Election(P),19510117,"[{'year': 2024, 'election_name': 'Presidential...",...,0.000000,0.000000,0.000000,1,0,0,0.00,0,0,0
24999,NY5a2bedd8-d4d6-,13850,19,3,DEM,20230801,A,2025 General Election(M);2025 Primary Election...,19790901,"[{'year': 2025, 'election_name': 'General Elec...",...,3.000000,0.400000,0.800000,1,0,0,1.00,0,0,0


In [29]:
#load model 
model = joblib.load(os.path.join('model', 'lightgbm_model.joblib'))

voter_data = X

# Make predictions
voter_data['turnout_probability'] = model.predict_proba(X[_])[:, 1]  # Get probabilities from model

In [30]:
voter_data

,SBOEID,RZIP5,ED,LD,ENROLLMENT,REGDATE,STATUS,voterhistory,DOB,parsed_history,...,avg_gap,gap_variability,longest_participation_streak,recent_pattern_change,participation_expansion,diversification_score,has_behavior_change,years_since_last_change,change_direction,turnout_probability
1,NY830cbd57-96ea-,13795,8,4,OTH,20200201,A,2024 General Election(P);2024 Primary Election...,19870508,"[{'year': 2024, 'election_name': 'General Elec...",...,1.333333,1.885618,1,1,0,1.00,1,2,1,0.000037
2,NY766abd27-a5b3-,13903,22,2,REP,19821215,A,2024 Presidential Primary Election(P);2022 Gen...,19360213,"[{'year': 2024, 'election_name': 'Presidential...",...,1.142857,0.989743,3,0,1,1.25,1,4,-1,0.023579
3,NYe437a174-dc88-,13760,31,3,OTH,20200326,I,2024 General Election(A),19830720,"[{'year': 2024, 'election_name': 'General Elec...",...,0.000000,0.000000,1,0,0,0.00,0,0,0,0.000023
4,NY34fe613f-a3e3-,13905,14,9,DEM,19950108,A,2013 General Election(P);2004 Primary Election...,19711003,"[{'year': 2013, 'election_name': 'General Elec...",...,6.000000,3.000000,1,1,1,2.00,1,13,1,0.000128
5,NYa447f596-42ba-,13901,12,4,BLK,20100707,A,2022 Primary Election(A),19360222,"[{'year': 2022, 'election_name': 'Primary Elec...",...,0.000000,0.000000,1,0,0,0.00,0,0,0,0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24996,NY278d9776-08e9-,13790,22,11,DEM,20200721,A,2024 General Election(P);2024 Primary Election...,19480115,"[{'year': 2024, 'election_name': 'General Elec...",...,1.000000,1.000000,1,1,0,1.00,0,0,0,0.000063
24997,NYa1c30a0a-4f53-,13790,24,10,REP,20060520,A,2022 General Election(A);2020 General Election...,19420319,"[{'year': 2022, 'election_name': 'General Elec...",...,3.200000,0.979796,1,1,0,1.00,1,5,1,0.000058
24998,NY778fd0c2-6aad-,13901,32,14,DEM,20241106,A,2024 Presidential Primary Election(P),19510117,"[{'year': 2024, 'election_name': 'Presidential...",...,0.000000,0.000000,1,0,0,0.00,0,0,0,0.000021
24999,NY5a2bedd8-d4d6-,13850,19,3,DEM,20230801,A,2025 General Election(M);2025 Primary Election...,19790901,"[{'year': 2025, 'election_name': 'General Elec...",...,0.400000,0.800000,1,0,0,1.00,0,0,0,0.998436


In [31]:
from segment import *


# Segment voters for campaign targeting
campaign_targets = segment_voters_for_campaign(voter_data)

# Export segment lists for field team
high_priority_canvass = campaign_targets[
    (campaign_targets['primary_tactic'] == 'CANVASS') & 
    (campaign_targets['targeting_score'] > 0.8)
].sort_values('targeting_score', ascending=False)

early_vote_targets = campaign_targets[
    campaign_targets['target_early_vote'] == 1
].sort_values(['segment', 'targeting_score'])


In [32]:
early_vote_targets

,SBOEID,RZIP5,ED,LD,ENROLLMENT,REGDATE,STATUS,voterhistory,DOB,parsed_history,...,diversification_score,has_behavior_change,years_since_last_change,change_direction,turnout_probability,segment,primary_tactic,targeting_score,target_early_vote,inconsistent_voter
11063,NY21470020-d39d-,13901,35,13,DEM,19940613,A,2021 General Election(E);2016 Primary Election...,19201011,"[{'year': 2021, 'election_name': 'General Elec...",...,1.00,1,5,1,0.792341,LIKELY,CANVASS,1.207659,1,1
12642,NY7374fbb6-c04b-,13904,4,3,REP,20000618,A,2022 General Election(A);2021 General Election...,19210506,"[{'year': 2022, 'election_name': 'General Elec...",...,0.75,1,4,1,0.763336,LIKELY,CANVASS,1.236664,1,1
14035,NYc9a6664e-1474-,13905,19,5,DEM,20170818,A,2021 General Election(E);2017 General Election(P),19710321,"[{'year': 2021, 'election_name': 'General Elec...",...,1.00,0,0,0,0.390839,PERSUADABLE,CANVASS,0.781678,1,1
22148,NY0ab586f7-adf7-,13905,29,8,BLK,19830803,A,2022 General Election(M);2022 Primary Election...,19331118,"[{'year': 2022, 'election_name': 'General Elec...",...,1.00,1,4,1,0.597132,PERSUADABLE,CANVASS,0.805737,1,1
3666,NY9b643f9e-aaac-,13760,20,12,BLK,20090626,A,2025 General Election(E);2017 General Election(P),19791021,"[{'year': 2025, 'election_name': 'General Elec...",...,1.00,0,0,0,0.409682,PERSUADABLE,CANVASS,0.819365,1,1
8970,NY8c1e1728-78fa-,13790,22,3,REP,20031118,A,2025 General Election(E);2017 General Election(M),19690628,"[{'year': 2025, 'election_name': 'General Elec...",...,1.00,0,0,0,0.409682,PERSUADABLE,CANVASS,0.819365,1,1
13475,NY4a1364c3-4b8d-,13901,28,14,BLK,19920817,A,2021 General Election(T);2015 General Election...,19140306,"[{'year': 2021, 'election_name': 'General Elec...",...,2.00,1,5,1,0.587014,PERSUADABLE,CANVASS,0.825971,1,1
3747,NYa887ffc8-3ea9-,13901,18,14,BLK,20050201,A,2021 General Election(E);2007 General Election...,19260313,"[{'year': 2021, 'election_name': 'General Elec...",...,2.00,1,5,1,0.422351,PERSUADABLE,CANVASS,0.844701,1,1
21848,NY5f08a8ba-aa8a-,13904,12,4,BLK,20211109,A,2021 Primary Election(E);2021 General Election(P),19690924,"[{'year': 2021, 'election_name': 'Primary Elec...",...,1.00,0,0,0,0.431444,PERSUADABLE,CANVASS,0.862888,1,0
10179,NY98aeff21-a2c2-,13901,14,11,DEM,20151109,A,2023 Primary Election(E);2021 General Election...,19600209,"[{'year': 2023, 'election_name': 'Primary Elec...",...,2.00,1,4,1,0.556045,PERSUADABLE,CANVASS,0.887910,1,1
